In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("studen.csv")
df.head()

,รหัสนักศึกษา,ชื่อ,อายุ,เพศ,คณะ,GPA,ส่วนสูง_cm,น้ำหนัก_kg,ชั่วโมงเรียน_ต่อสัปดาห์,วันที่ลงทะเบียน
0,64001,สมชาย,20.0,ชาย,วิทยาศาสตร์,3.21,170.0,65.0,15.0,2024-01-24
1,64002,สมหญิง,21.0,หญิง,วิทย์,3.45,165.0,58.0,20.0,2023-12-19
2,64003,NaN,NaN,Male,Science,3.67,175.0,NaN,25.0,2023-12-17
3,64004,สมศักดิ์,22.0,ช,คณะวิทยาศาสตร์,-1.00,168.0,70.0,10.0,2023-12-21
4,64005,สมใจ,19.0,ญ,วิศวะ,3.89,160.0,52.0,30.0,2024-02-07


In [21]:
# 1. ลบช่องว่างหน้า-หลังข้อความทุกคอลัมน์ที่เป็น string
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [22]:
# 2. แก้ค่าที่ไม่สอดคล้องกันในคอลัมน์ "เพศ"
gender_map = {
    'ชาย': 'ชาย',
    'ช': 'ชาย',
    'Male': 'ชาย',
    'หญิง': 'หญิง',
    'ญ': 'หญิง',
    'Female': 'หญิง'
}
df['เพศ'] = df['เพศ'].replace(gender_map)

In [23]:
# 3. จัดการค่าที่ผิดปกติใน GPA (เช่น ค่า < 0 หรือ > 4)
df.loc[(df['GPA'] < 0) | (df['GPA'] > 4), 'GPA'] = np.nan

In [24]:
# 4. เติม missing values
# อายุ: ใช้ค่าเฉลี่ย
df['อายุ'] = df['อายุ'].fillna(df['อายุ'].mean())
# น้ำหนัก: ใช้ค่าเฉลี่ย
df['น้ำหนัก_kg'] = df['น้ำหนัก_kg'].fillna(df['น้ำหนัก_kg'].mean())
# ชื่อ: ถ้าไม่มีชื่อให้ใส่ "ไม่ทราบ"
df['ชื่อ'] = df['ชื่อ'].fillna('ไม่ทราบ')
# คณะ: ใช้ forward fill
df['คณะ'] = df['คณะ'].fillna(method='ffill')

In [25]:
# 5. แปลงวันที่ให้เป็น datetime
df['วันที่ลงทะเบียน'] = pd.to_datetime(df['วันที่ลงทะเบียน'], errors='coerce')

In [26]:
# 6. ตรวจสอบผลลัพธ์หลังทำความสะอาด
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   รหัสนักศึกษา             20 non-null     int64         
 1   ชื่อ                     20 non-null     object        
 2   อายุ                     20 non-null     float64       
 3   เพศ                      18 non-null     object        
 4   คณะ                      20 non-null     object        
 5   GPA                      16 non-null     float64       
 6   ส่วนสูง_cm               19 non-null     float64       
 7   น้ำหนัก_kg               20 non-null     float64       
 8   ชั่วโมงเรียน_ต่อสัปดาห์  18 non-null     float64       
 9   วันที่ลงทะเบียน          20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 1.7+ KB
None
   รหัสนักศึกษา      ชื่อ       อายุ   เพศ             คณะ   GPA  ส่วนสูง_cm  \

In [30]:
df['ส่วนสูง_m'] = df['ส่วนสูง_cm'] / 100
df['BMI'] = df['น้ำหนัก_kg'] / (df['ส่วนสูง_m'] ** 2)

def bmi_category(bmi):
    if bmi < 18.5:
        return 'น้ำหนักน้อย'
    elif bmi < 23:
        return 'ปกติ'
    elif bmi < 25:
        return 'น้ำหนักเกิน'
    else:
        return 'อ้วน'
df['BMI_Category'] = df['BMI'].apply(bmi_category)

df['Age_Group'] = pd.cut(df['อายุ'],
                         bins=[0, 18, 22, 25, 30, np.inf],
                         labels=['<=18', '19-22', '23-25', '26-30', '30+'])

df['Register_Week'] = df['วันที่ลงทะเบียน'].dt.isocalendar().week

df['GPA_Level'] = pd.cut(df['GPA'],
                         bins=[0, 2, 3, 3.5, 4],
                         labels=['ต่ำ', 'ปานกลาง', 'ดี', 'ดีมาก'])

df['ชั่วโมงเรียน_ต่อปี'] = df['ชั่วโมงเรียน_ต่อสัปดาห์'] * 52

df['Gender_Code'] = df['เพศ'].map({'ชาย': 0, 'หญิง': 1})

In [31]:

print(df.head())

   รหัสนักศึกษา      ชื่อ       อายุ   เพศ             คณะ   GPA  ส่วนสูง_cm  \
0         64001     สมชาย  20.000000   ชาย     วิทยาศาสตร์  3.21       170.0   
1         64002    สมหญิง  21.000000  หญิง           วิทย์  3.45       165.0   
2         64003   ไม่ทราบ  28.470588   ชาย         Science  3.67       175.0   
3         64004  สมศักดิ์  22.000000   ชาย  คณะวิทยาศาสตร์   NaN       168.0   
4         64005      สมใจ  19.000000  หญิง           วิศวะ  3.89       160.0   

   น้ำหนัก_kg  ชั่วโมงเรียน_ต่อสัปดาห์ วันที่ลงทะเบียน  ส่วนสูง_m        BMI  \
0   65.000000                     15.0      2024-01-24       1.70  22.491349   
1   58.000000                     20.0      2023-12-19       1.65  21.303949   
2   63.611111                     25.0      2023-12-17       1.75  20.770975   
3   70.000000                     10.0      2023-12-21       1.68  24.801587   
4   52.000000                     30.0      2024-02-07       1.60  20.312500   

  BMI_Category Age_Group  Register_Wee

In [32]:
# ดูข้อมูลเบื้องต้น
print(df.info())
print(df.describe())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   รหัสนักศึกษา             20 non-null     int64         
 1   ชื่อ                     20 non-null     object        
 2   อายุ                     20 non-null     float64       
 3   เพศ                      18 non-null     object        
 4   คณะ                      20 non-null     object        
 5   GPA                      16 non-null     float64       
 6   ส่วนสูง_cm               19 non-null     float64       
 7   น้ำหนัก_kg               20 non-null     float64       
 8   ชั่วโมงเรียน_ต่อสัปดาห์  18 non-null     float64       
 9   วันที่ลงทะเบียน          20 non-null     datetime64[ns]
 10  ส่วนสูง_m                19 non-null     float64       
 11  BMI                      19 non-null     float64       
 12  BMI_Category             20 non-null  

In [34]:
# สร้าง log เก็บประวัติการแก้ไข
changes_log = []

# 1. แทนค่า GPA ที่เป็น -1 ด้วย NaN
df.loc[df['GPA'] < 0, 'GPA'] = np.nan
changes_log.append("แทนค่า GPA ที่เป็น -1 ด้วย NaN")

# 2. เติมค่า GPA ที่หายไปด้วยค่าเฉลี่ย
mean_gpa = df['GPA'].mean()
df['GPA'] = df['GPA'].fillna(mean_gpa)
changes_log.append(f"เติม GPA ที่หายด้วยค่าเฉลี่ย {mean_gpa:.2f}")

# 3. แสดง log
for change in changes_log:
    print("-", change)

- แทนค่า GPA ที่เป็น -1 ด้วย NaN
- เติม GPA ที่หายด้วยค่าเฉลี่ย 3.43


In [35]:
# สร้าง log การแก้ไข
changes_log = []
changes_log.append("แทนค่า GPA -1 ด้วย NaN")
changes_log.append(f"เติม GPA ที่หายด้วยค่าเฉลี่ย {mean_gpa:.2f}")

In [37]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'A': [1, 2, np.nan],
    'B': [4, 5, 6]
})

# เติมค่า A ที่หาย
df['A'] = df['A'].fillna(df['A'].mean())

# ตรวจสอบว่าไม่มี NaN เหลือ
assert df.isnull().sum().sum() == 0, "ยังมีค่าหายอยู่!"
print("ผ่านการตรวจสอบ: ไม่มีค่าหาย")


ผ่านการตรวจสอบ: ไม่มีค่าหาย


In [38]:
print(df.isnull().sum())
assert df.isnull().sum().sum() == 0, "ยังมีค่าหายอยู่!"

A    0
B    0
dtype: int64
